<a href="https://colab.research.google.com/github/KKKKKIKKKK/BaekJoon.java/blob/master/Easy_Fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

os.getcwd()

'/content'

In [4]:
%cd drive/MyDrive/

/content/drive/MyDrive


In [5]:
!pip install -q -U bitsandbytes
!pip install datasets -U
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install pandas
!pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which 

In [6]:

from datasets import load_dataset
import pandas as pd

In [7]:
# 모델 레포지토리
model_path = "microsoft/Phi-3.5-mini-instruct"

# 데이터 path
data_path = "kyujinpy/KOpen-platypus"

In [8]:
# dataset 다운

data = load_dataset(data_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/24926 [00:00<?, ? examples/s]

In [9]:
df = pd.DataFrame(data['train'])

In [10]:
# 데이터셋 구성확인
df.head()

,input,output,instruction,data_source
0,,"모든 가능한 결과의 확률의 합이 1$이므로, 스피너가 $C$에 착륙할 확률을 구하려...","보드 게임 스피너는 $A$, $B$, $C$로 표시된 세 부분으로 나뉩니다. 스피너...",MATH/PRM-800K
1,,14명 중 6명을 선택해야 하는데 순서는 중요하지 않습니다. 이것은 순열 문제가 아...,저희 학교 수학 클럽에는 남학생 6명과 여학생 8명이 있습니다. 주 수학 경시대회...,MATH/PRM-800K
2,,먼저 단어에 제한을 두지 않고 4글자로 된 모든 단어의 개수를 세어봅니다. 그런 다...,"자음이 하나 이상인 4글자 단어는 $A$, $B$, $C$, $D$, $E$로 몇 ...",MATH/PRM-800K
3,,주사위 중 하나 이상이 1에 나올 때만 가능합니다. 두 주사위가 모두 1이 아닐 확...,멜린다는 표준 6면 주사위 두 개를 굴려서 굴린 두 개의 숫자로 두 자리 숫자를 만...,MATH/PRM-800K
4,,문제를 H와 T의 시퀀스라고 생각하세요. 두 개의 T가 연속으로 나타날 수 없으므로...,p$를 공정한 동전을 반복적으로 던지는 과정에서 5$의 앞면이 나오기 전에 2$의 ...,MATH/PRM-800K


In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, Trainer, TrainingArguments, DataCollatorForSeq2Seq
import bitsandbytes as bnb
from huggingface_hub import login
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training)


In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [13]:

bos = tokenizer.bos_token_id
eos = tokenizer.eos_token_id
pad = tokenizer.pad_token_id

tokenizer.pad_token_id = eos
tokenizer.padding_side = "right"
cut_off_len = 2048
val_size = 0.05


In [14]:

prompt_input = "아래는 작업을 설명하는 지시 사항과 추가적인 문맥을 제공하는 입력이 함께 제공된 것입니다. 요청을 적절하게 완료하는 응답을 작성하세요.\n\n### 지시사항:\n{instruction}\n\n### 입력:\n{input}\n\n### 응답:\n"
prompt_no_input = "아래는 작업을 설명하는 지시 사항입니다. 요청을 적절하게 완료하는 응답을 작성하세요.\n\n### 지시사항:\n{instruction}\n\n### 응답:\n"

In [15]:
train_dataset  = data.map(
    lambda x: {
        'text': (
            f"{prompt_input.format(instruction=x['instruction'], input=x['input'])}{x['output']}{tokenizer.eos_token}"
            if 'input' in x and x['input'].strip() else
            f"{prompt_no_input.format(instruction=x['instruction'])}{x['output']}{tokenizer.eos_token}"
            )
        }
    )

Map:   0%|          | 0/24926 [00:00<?, ? examples/s]

In [16]:
print(train_dataset)

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction', 'data_source', 'text'],
        num_rows: 24926
    })
})


In [17]:
val_set_size=0.05

In [18]:
if val_set_size > 0:
      train_val = train_dataset["train"].train_test_split(
            test_size=val_set_size, shuffle=True, seed=42
        )
      train_data = (
            train_val["train"]
        )
      val_data = (
            train_val["test"]
        )
else:
      train_data = data["train"]
      val_data = None

In [19]:
train_data

Dataset({
    features: ['input', 'output', 'instruction', 'data_source', 'text'],
    num_rows: 23679
})

In [20]:
train_data = train_data.map(lambda samples: tokenizer(samples["text"]), batched=True)

Map:   0%|          | 0/23679 [00:00<?, ? examples/s]

In [21]:
print(train_data)

Dataset({
    features: ['input', 'output', 'instruction', 'data_source', 'text', 'input_ids', 'attention_mask'],
    num_rows: 23679
})


In [22]:
train_data = train_data.map(
    lambda samples: {
        'input_ids': samples['input_ids'],
        'attention_mask': samples['attention_mask'],
        'labels': samples['input_ids']  # labels 필드 추가
    },
    batched=True
)

Map:   0%|          | 0/23679 [00:00<?, ? examples/s]

In [23]:
print(train_data)

Dataset({
    features: ['input', 'output', 'instruction', 'data_source', 'text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 23679
})


In [24]:
val_data = val_data.map(lambda samples: tokenizer(samples["text"]), batched=True)

Map:   0%|          | 0/1247 [00:00<?, ? examples/s]

In [25]:
val_data = val_data.map(
    lambda samples: {
        'input_ids': samples['input_ids'],
        'attention_mask': samples['attention_mask'],
        'labels': samples['input_ids']  # labels 필드 추가
    },
    batched=True)

Map:   0%|          | 0/1247 [00:00<?, ? examples/s]

In [26]:
# train_data=train_data.remove_columns(['input', 'output', 'instruction', 'data_source', 'text'])

# val_data=val_data.remove_columns(['input', 'output', 'instruction', 'data_source', 'text'])

In [33]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_storage=torch.bfloat16,
    )


In [34]:
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config = quantization_config,
    torch_dtype = torch.bfloat16,
    device_map = {"" : 0}
    )


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [35]:


model.config.use_cache = False
model = prepare_model_for_kbit_training(model)

model.gradient_checkpointing_enable()

In [37]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha = 16,
    target_modules = ['q_proj', 'k_proj', 'v_proj','o_proj',],
    lora_dropout = 0.05,
    bias = "none",
    task_type = "CAUSAL_LM"
)

model = get_peft_model(model, config)

In [38]:
#Hyper parameter setting

output_dir='./easy_train'


num_epochs = 1
micro_batch_size = 2
gradient_accumulation_steps = 1
warmup_steps = 10
learning_rate = 1e-7
group_by_length = False
optimizer = 'paged_adamw_8bit'

# adam 활용시
beta1 = 0.9
beta2 = 0.95

lr_scheduler = 'cosine'

use_wandb = True
wandb_run_name = 'Single_GPU_Optim'

use_fp16 = False
use_bf_16 = True

evaluation_strategy = 'steps'
eval_steps = 20
save_steps = 20
save_strategy = 'steps'

In [40]:
trainer = Trainer(
    model=model,

    train_dataset=train_data,
    eval_dataset=val_data,
    args =
    TrainingArguments(
        max_steps=100,
        per_device_train_batch_size = micro_batch_size,
        gradient_accumulation_steps = gradient_accumulation_steps,
        per_device_eval_batch_size = micro_batch_size,
        warmup_steps = warmup_steps,
        num_train_epochs = num_epochs,
        learning_rate = learning_rate,
        adam_beta1 = beta1, # adam 활용할때 사용
        adam_beta2 = beta2, # adam 활용할때 사용
        fp16 = use_fp16,
        bf16 = use_bf_16,
        logging_steps = 1,
        optim = optimizer,
        evaluation_strategy = evaluation_strategy if val_size > 0 else "no",
        save_strategy="steps",  #스텝기준으로 save
        eval_steps = eval_steps if val_size > 0 else None,
        save_steps = save_steps,
        lr_scheduler_type=lr_scheduler,
        output_dir = output_dir,
        load_best_model_at_end = True if val_size > 0 else False ,
        group_by_length=group_by_length,
        report_to="wandb" if use_wandb else None,
        run_name=wandb_run_name if use_wandb else None,

        ),
    data_collator=DataCollatorForSeq2Seq(tokenizer,padding=True),
    )

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1539: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [41]:
trainer.model.print_trainable_parameters()

trainable params: 1,572,864 || all params: 3,822,652,416 || trainable%: 0.0411


In [ ]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss


In [ ]:
tokenizer.save_pretrained(output_dir)
trainer.save_model()

In [ ]:
!huggingface-cli login #enter your Hugging face Toke

In [ ]:
model.push_to_hub('허깅페이스아이디/내가 짓고 싶은 모델 이름')
tokenizer.push_to_hub('허깅페이스아이디/내가 짓고 싶은 모델 이름')